In [148]:
import backtrader as bt
import pandas as pd
from datetime import datetime,timedelta

class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 6),
    )
    def log(self, txt, dt=None):
        ''' 记录'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # 增加移动平均指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        
        # 增加划线的指标
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        flag = True
        # flag=stock_hfq_df.loc[str(self.datas[0].datetime.date(0))].Value_pre>stock_hfq_df.loc[str(self.datas[0].datetime.date(-1))].Value_pre
        # flag = True
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...SMA均线
            if self.dataclose[0] > self.sma[0] and flag:
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                
                   
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0] and not flag:#SMA均线
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [149]:
#获取数据
# stock_hfq_df = pd.read_excel("PRE_BCHAIN.xlsx",index_col='Date',parse_dates=True)
stock_hfq_df = pd.read_excel("Bitcoin.xlsx",index_col='date',parse_dates=True)
# print(stock_hfq_df)
# print(stock_hfq_df[0:100])
start_date = datetime(2016, 11, 12)  # 回测开始时间
end_date = datetime(2021, 2, 12)  # 回测结束时间
# end_date = start_date + timedelta(days=60)
pre=stock_hfq_df.Value_pre
# Close	Low	High	Open	Volume

In [150]:
money = 0

In [151]:

cerebro = bt.Cerebro()
# 增加一个策略
cerebro.addstrategy(TestStrategy)
# 将数据传入回测系统
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)
cerebro.broker.setcash(1000)

cerebro.broker.setcommission(commission=0.02)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000.00
2016-12-15, Close, 776.75
2016-12-15, BUY CREATE, 776.75
2016-12-16, BUY EXECUTED, Price: 776.75, Cost: 776.75, Comm 15.54
2016-12-16, Close, 775.88
2016-12-17, Close, 788.70
2016-12-18, Close, 788.40
2016-12-19, Close, 788.67
2016-12-20, Close, 793.09
2016-12-21, Close, 824.21
2016-12-22, Close, 834.97
2016-12-23, Close, 901.31
2016-12-24, Close, 891.61
2016-12-25, Close, 892.60
2016-12-26, Close, 897.33
2016-12-27, Close, 930.37
2016-12-28, Close, 930.34
2016-12-29, Close, 963.38
2016-12-30, Close, 952.15
2016-12-31, Close, 958.12
2017-01-01, Close, 997.72
2017-01-02, Close, 1015.97
2017-01-03, Close, 1013.42
2017-01-04, Close, 1126.76
2017-01-05, Close, 994.67
2017-01-06, Close, 999.65
2017-01-07, Close, 896.83
2017-01-08, Close, 908.14
2017-01-09, Close, 910.49
2017-01-10, Close, 906.05
2017-01-11, Close, 785.22
2017-01-12, Close, 780.92
2017-01-13, Close, 826.29
2017-01-14, Close, 817.91
2017-01-15, Close, 819.55
2017-01-16, Close, 830.50
2017-01-